# Welcome to the Day 2 Lab!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get started --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## First - let's talk about the Chat Completions API

1. The simplest way to call an LLM
2. It's called Chat Completions because it's saying: "here is a conversation, please predict what should come next"
3. The Chat Completions API was invented by OpenAI, but it's so popular that everybody uses it!

### We will start by calling OpenAI again - but don't worry non-OpenAI people, your time is coming!


In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


## Do you know what an Endpoint is?

If not, please review the Technical Foundations guide in the guides folder

And, here is an endpoint that might interest you...

In [2]:
import requests

headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

payload = {
    "model": "gpt-5-nano",
    "messages": [
        {"role": "user", "content": "Tell me a fun fact"}]
}

payload

{'model': 'gpt-5-nano',
 'messages': [{'role': 'user', 'content': 'Tell me a fun fact'}]}

In [3]:
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=payload
)

response.json()

{'id': 'chatcmpl-CtZp8Zy6L2SF2KsIbXo9DtvSPn2Q9',
 'object': 'chat.completion',
 'created': 1767361782,
 'model': 'gpt-5-nano-2025-08-07',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Fun fact: A day on Venus is longer than its year. Venus takes about 243 Earth days to rotate once, but only about 225 Earth days to orbit the Sun. So a single day on Venus is longer than a whole year there. Want another fun fact?',
    'refusal': None,
    'annotations': []},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 11,
  'completion_tokens': 704,
  'total_tokens': 715,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 640,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': None}

In [4]:
response.json()["choices"][0]["message"]["content"]

'Fun fact: A day on Venus is longer than its year. Venus takes about 243 Earth days to rotate once, but only about 225 Earth days to orbit the Sun. So a single day on Venus is longer than a whole year there. Want another fun fact?'

# What is the openai package?

It's known as a Python Client Library.

It's nothing more than a wrapper around making this exact call to the http endpoint.

It just allows you to work with nice Python code instead of messing around with janky json objects.

But that's it. It's open-source and lightweight. Some people think it contains OpenAI model code - it doesn't!


In [5]:
# Create OpenAI client

from openai import OpenAI
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content



'Fun fact: Wombats poop cubes. Their long, slow digestion and the way water is absorbed in their intestines shape the droppings into cubes, which helps them stack and not roll away—great for marking territory. Want another fun fact?'

## And then this great thing happened:

OpenAI's Chat Completions API was so popular, that the other model providers created endpoints that are identical.

They are known as the "OpenAI Compatible Endpoints".

For example, google made one here: https://generativelanguage.googleapis.com/v1beta/openai/

And OpenAI decided to be kind: they said, hey, you can just use the same client library that we made for GPT. We'll allow you to specify a different endpoint URL and a different key, to use another provider.

So you can use:

```python
gemini = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key="AIz....")
gemini.chat.completions.create(...)
```

And to be clear - even though OpenAI is in the code, we're only using this lightweight python client library to call the endpoint - there's no OpenAI model involved here.

If you're confused, please review Guide 9 in the Guides folder!

And now let's try it!

## THIS IS OPTIONAL - but if you wish to try out Google Gemini, please visit:

https://aistudio.google.com/

And set up your API key at

https://aistudio.google.com/api-keys

And then add your key to the `.env` file, being sure to Save the .env file after you change it:

`GOOGLE_API_KEY=AIz...`


In [7]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

load_dotenv(override=True)

google_api_key = os.getenv("GEMINI_API_KEY")

if not google_api_key:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
elif not google_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")



API key found and looks good so far!


In [8]:
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

response = gemini.chat.completions.create(model="gemini-2.5-flash-lite", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

'Here\'s a fun fact for you:\n\n**A group of pugs is called a "grumble."**\n\nIsn\'t that just the cutest thing? Imagine a whole bunch of pugs making their happy, snorty grumbles!'

## And Ollama also gives an OpenAI compatible endpoint

...and it's on your local machine!

If the next cell doesn't print "Ollama is running" then please open a terminal and run `ollama serve`

In [ ]:
requests.get("http://localhost:11434").content

### Download llama3.2 from meta

Change this to llama3.2:1b if your computer is smaller.

Don't use llama3.3 or llama4! They are too big for your computer..

In [ ]:
!ollama pull llama3.2

In [9]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')


In [10]:
# Get a fun fact

response = ollama.chat.completions.create(model="llama3.2", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

"Here's one:\n\nDid you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible! Honey's unique composition, which includes antioxidants and an acid-neutralizing pH level, makes it a virtually self-preserving food. Isn't that sweet?"

In [11]:
# Now let's try deepseek-r1:1.5b - this is DeepSeek "distilled" into Qwen from Alibaba Cloud

!ollama pull deepseek-r1:1.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 1.6 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏ 8.7 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏  12 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   2% ▕                  ▏  19 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   2% ▕                  ▏  25 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   3% ▕                  ▏  28 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   3% ▕                  ▏  36 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   4% ▕                  ▏  44 MB/1.1 GB              

In [12]:
response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

"Sure! Here's a fun puzzle:\n\n**Puzzle:** Can you solve this puzzle? Think about time travel and the past. If you're up for it, think carefully before answering.\n\n[ A picture of a time machine or someone traveling through time appears in the background ]\n\nSo, what did you think? Now, do you want to try solving any other puzzle I've given?"

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [ ]:
import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI


load_dotenv()

openAi = OpenAI(base_url=os.getenv("OLLAMA_BASE_URL"))


def generate_message(url: str) -> list[dict[str, str]]:
    content = fetch_website_contents(url)
    messages = [
        {
            "role": "system",
            "content": """
                    You are a helpful assistant that summarizes website content. The response should be concise and easy to read.
                    Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
                """,
        },
        {
            "role": "user",
            "content": "Give me the highlights of the today's news, repond in Spanish. The content of it is "+ content,
        },
    ]
    return messages

def call_llm(messages: list[dict[str, str]]) -> str:
    response = openAi.chat.completions.create(
        model="deepseek-r1",
        messages=messages,
    )
    return response.choices[0].message.content
    

def main():
    url = "https://20minutos.es"
    messages = generate_message(url)
    print(messages)
    summary = call_llm(messages)
    display(Markdown(summary))

main()

[{'role': 'system', 'content': '\n                    You are a helpful assistant that summarizes website content. The response should be concise and easy to read.\n                    Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n                '}, {'role': 'user', 'content': "Give me the highlights of the today's news, repond in Spanish. The content of it is 20minutos.es | Diario abierto | Última hora, local, España y el mundo\n\n20minutos\nEdiciones\nESPAÑA\nCATALUÑA\nMADRID\nIniciar sesión\nMi perfil\nSalir\nIniciar sesión\nEDICIONES\nMadrid\nBarcelona\nValencia\nAndalucía / Sevilla\n20MINUTOS EN PAPEL\nLee y descarga el periódico\nsecciones\nNacional\nInternacional\nLa Información Económica\nOpinión\nGente\nTelevisión\nDeportes\nCultura\nCiencia\nCapaces\nAnimaleros\nGonzoo\nMi Bolsillo\nBlogs\nverticales\nSalud\nGastronomía\nTecnología\nViajes\nMotor\nnewsletters\nÚltima hora\n20AM\nVersión impresa\nMi bolsillo\nOtras newsletter